<h1>Putting on my chemist robe and glasses :)</h1><center><img src="https://static.displate.com/280x392/displate/2022-12-28/494bd2c86cb966c2129375457a82982f_80fd640e09154845b9ca853fda67a96d.jpg" ></center>Let us first figure out what they want from us and what we want from our code. <br><br>


The dataset comprises binary classification data representing whether small molecules bind to three different protein targets. Each entry includes SMILES representations of molecule structures and binary labels for binding to each protein target. The competition data, provided by Leash Biosciences, consists of approximately **98M** training examples per protein, **200K** validation examples per protein, and **360K** test molecules per protein. *Keep in mind that dataset is very imbalanced:  roughly 0.5% of examples are classified as binders*<br><br>

<center><h2>So, what is SMILES?</h2><img src="https://static.wixstatic.com/media/7cced3_08c22a1ad56a4d2b81c46c8d71ebc34b~mv2.gif" ></center><br>

SMILES (Simplified Molecular Input Line Entry System) is a notation system for representing chemical structures in a computer-readable format. Developed with funding from the U.S. Environmental Protection Agency, it offers a flexible and easily learned approach to representing molecules. SMILES notation follows five basic syntax rules:

* Atoms and Bonds: Atoms are represented by their atomic symbols, with lowercase letters indicating aromatic atoms. Bonds are denoted by symbols (- for single, = for double, # for triple, * for aromatic, and . for disconnected structures).
* Simple Chains: Chains of atoms are represented by combining atomic symbols and bond symbols. Hydrogen atoms are suppressed unless explicitly stated.
* Branches: Branches from chains are enclosed in parentheses and placed directly after the atom to which they are connected.
* Rings: Ring structures are identified by using numbers to denote the opening and closing ring atoms. Different numbers are used for each ring, and bond symbols may precede the ring closure number.
* Charged Atoms: Charges on atoms are indicated by placing the atom symbol within brackets, enclosing the charge.<br>


<center><h2>What are the targets?</h2></center><br>

The dataset includes three protein targets:
<h3>EPHX2 (sEH):</h3><center><img src="https://www.researchgate.net/publication/26836510/figure/fig1/AS:394310370512898@1471022326590/Pathways-of-EETs-synthesis-metabolism-and-action.png" ></center><br>
* This target refers to epoxide hydrolase 2, encoded by the EPHX2 genetic locus. Its protein product, commonly known as soluble epoxide hydrolase (sEH), is an enzyme that catalyzes certain chemical reactions and hydrolyzes phosphate groups. It is a potential drug target for conditions like high blood pressure and diabetes. The dataset includes screening data obtained from Leash Biosciences, along with structural information for model evaluation.
<h3>BRD4:</h3><center><img src="https://ars.els-cdn.com/content/image/1-s2.0-S1043661823001238-ga1.jpg" ></center><br>
* Bromodomain 4 is encoded by the BRD4 locus. Its protein product, also named BRD4, plays a role in gene transcription regulation by binding to histones in the nucleus. This protein is implicated in cancer progression, and inhibiting its activity has been explored as a therapeutic strategy. The dataset includes screening data from Leash Biosciences and structural information for model evaluation.
<h3>ALB (HSA):</h3><center><img src="https://ars.els-cdn.com/content/image/1-s2.0-S0162013419301734-gr4.jpg" ></center><br>
* Serum albumin, encoded by the ALB locus, is the most abundant protein in blood. It regulates osmotic pressure and transports various molecules, including drugs, hormones, and fatty acids. Predicting the binding of small molecules to albumin is crucial for drug development, as it impacts drug distribution and effectiveness. The dataset includes screening data from Leash Biosciences and structural information for model evaluation.


<h3>Now let's get the things done! :D</h3> <br>

We shall start with the installation of necessary tools. <br><br>
**DuckDB is an open-source, in-memory database system optimized for fast analytical querying and low memory usage. It's lightweight, embeddable, and seamlessly integrates with popular programming languages like Python and R. DuckDB excels in executing rapid SQL queries on large datasets, making it ideal for data exploration and interactive analysis tasks.**

In [1]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 47.4 MB/s eta 0:00:00


**RDKit is a widely-used open-source toolkit for cheminformatics. It offers various functions for working with chemical structures and data, including molecular representation, substructure searching, and compound library handling. RDKit is popular in pharmaceutical research and drug discovery due to its comprehensive features and easy integration with Python.**

In [2]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 37.1 MB/s eta 0:00:00


In [3]:
# !pip install tensorflow

In [4]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import duckdb
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import average_precision_score

# nn
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import os
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping


2024-04-22 21:35:03.545896: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-22 21:35:03.546034: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-22 21:35:03.733277: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
%%time
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 30000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 30000)""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 2min 50s, sys: 10.2 s, total: 3min
Wall time: 54.4 s


In [6]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,18392516,CC(C)CC(NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,Nc1ccc2c(c1)OCCCO2,CN(C)C(=O)C1CCC(CN)O1.Cl,CC(C)CC(Nc1nc(NCC2CCC(C(=O)N(C)C)O2)nc(Nc2ccc3...,sEH,0
1,130157979,O=C(N[C@H]1CCC[C@@H]1C(=O)O)OCC1c2ccccc2-c2ccc...,Cl.Cl.NCc1nc2cnccc2s1,COc1ncc(N)cn1,COc1ncc(Nc2nc(NCc3nc4cnccc4s3)nc(N[C@H]3CCC[C@...,BRD4,0
2,238350683,O=C(O)CC1(CNC(=O)OCC2c3ccccc3-c3ccccc32)CCCCC1,Nc1cc(Br)ccc1CO,Nc1ccc(N2CCC(N3CCOCC3)CC2)cc1,O=C(CC1(CNc2nc(Nc3ccc(N4CCC(N5CCOCC5)CC4)cc3)n...,sEH,0
3,68166565,N#Cc1ccc(NC(=O)OCC2c3ccccc3-c3ccccc32)c(C(=O)O)c1,Cl.NCCNC(N)=O,NCc1nccs1,N#Cc1ccc(Nc2nc(NCCNC(N)=O)nc(NCc3nccs3)n2)c(C(...,HSA,0
4,57929930,Cc1ccc(NC(=O)OCC2c3ccccc3-c3ccccc32)c(C(=O)O)c1,COC(=O)c1ccc(N)cc1OC,NCc1cccc(C(F)(F)F)n1,COC(=O)c1ccc(Nc2nc(NCc3cccc(C(F)(F)F)n3)nc(Nc3...,sEH,0


Further, we convert SMILES representations of molecules into RDKit molecule objects and then generates ECFP fingerprints for each molecule, storing the results in the DataFrame. 

**ECFP, or Extended Connectivity Fingerprints**, are molecular fingerprints used in cheminformatics to encode structural features of molecules. They represent connectivity patterns of atoms by hashing local environments within a specified radius. ECFP fingerprints are widely used for similarity searching and activity prediction in drug discovery and cheminformatics due to their efficiency and robustness.

<center><img src="https://miro.medium.com/max/652/1*YjmaYuWldV2yFH1TvPvNNw.jpeg" ></center><br>

In [7]:
%%time
# Convert SMILES to RDKit molecules
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)

# Generate ECFPs
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

df['ecfp'] = df['molecule'].apply(generate_ecfp)

CPU times: user 1min 34s, sys: 2.17 s, total: 1min 36s
Wall time: 1min 36s


In [8]:
df['ecfp'].head()

0    [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...
1    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2    [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
4    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: ecfp, dtype: object

In [9]:
RANDOM_STATE = 42

In [10]:
%%time
# One-hot encode the protein_name
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

# Combine ECFPs and one-hot encoded protein_name
X = [ecfp + protein for ecfp, protein in zip(df['ecfp'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

# Assume 'X' is a list of combined ECFP and one-hot encoded data
X = np.array(X)  # Convert the entire X list to a NumPy array
y = np.array(y)  # Also convert y if it's not already an array

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

CPU times: user 11.4 s, sys: 804 ms, total: 12.2 s
Wall time: 12.2 s


In [11]:
# NUMBER_OF_MODELS = 3
# # Create and train the ensemble models
# catboost_model = CatBoostClassifier(iterations=500, random_state=RANDOM_STATE)
# lgbm_model = LGBMClassifier(random_state=RANDOM_STATE)
# xgb_model = XGBClassifier(random_state=RANDOM_STATE)

# catboost_model.fit(X_train, y_train)
# lgbm_model.fit(X_train, y_train)
# xgb_model.fit(X_train, y_train)

# # Make predictions on the test set
# y_pred_proba_catboost = catboost_model.predict_proba(X_test)[:, 1]  # Probability of the positive class
# y_pred_proba_lgbm = lgbm_model.predict_proba(X_test)[:, 1]  # Probability of the positive class
# y_pred_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# # Average the predictions from the ensemble
# y_pred_proba_ensemble = (y_pred_proba_catboost + y_pred_proba_lgbm + y_pred_proba_xgb) / NUMBER_OF_MODELS

# # Calculate the mean average precision
# map_score = average_precision_score(y_test, y_pred_proba_ensemble)
# print(f"Mean Average Precision (mAP): {map_score:.8f}")


NUMBER_OF_MODELS = 4
# # Neural Network Setup
# model = Sequential([
#     Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
#     Dropout(0.5),
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(1, activation='sigmoid')
# ])
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # Train the neural network
# model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


model = Sequential([
    Dense(1024, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

# Ensemble Model Setup
catboost_model = CatBoostClassifier(iterations=500, random_state=42)
lgbm_model = LGBMClassifier(random_state=42)
xgb_model = XGBClassifier(random_state=42)

catboost_model.fit(X_train, y_train)
lgbm_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)

# Predictions
y_pred_proba_catboost = catboost_model.predict_proba(X_test)[:, 1]
y_pred_proba_lgbm = lgbm_model.predict_proba(X_test)[:, 1]
y_pred_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]
y_pred_proba_nn = model.predict(X_test).flatten()

# Average the predictions from the ensemble
y_pred_proba_ensemble = (y_pred_proba_catboost + y_pred_proba_lgbm + y_pred_proba_xgb + y_pred_proba_nn) / 4

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba_ensemble)
print(f"Mean Average Precision (mAP): {map_score:.8f}")

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 18s 27ms/step - accuracy: 0.7890 - loss: 0.4337 - val_accuracy: 0.8842 - val_loss: 0.2819
Epoch 2/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.8990 - loss: 0.2469 - val_accuracy: 0.9031 - val_loss: 0.2442
Epoch 3/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - accuracy: 0.9258 - loss: 0.1869 - val_accuracy: 0.9024 - val_loss: 0.2408
Epoch 4/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - accuracy: 0.9404 - loss: 0.1476 - val_accuracy: 0.9046 - val_loss: 0.2457
Epoch 5/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 14s 24ms/step - accuracy: 0.9533 - loss: 0.1189 - val_accuracy: 0.9093 - val_loss: 0.2662
Epoch 6/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 21s 25ms/step - accuracy: 0.9630 - loss: 0.0960 - val_accuracy: 0.9077 - val_loss: 0.2955
Epoch 7/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 21s 25ms/step - accuracy: 0.9695 - loss: 0.0784 - val_accuracy: 0.9053 - val_loss: 0.3265
Epoch 8/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 15s 25ms/step - accuracy: 0.9769 - loss: 0

In [12]:
import os
# Process the test.parquet file chunk by chunk sinc the file is huge
test_file = '/kaggle/input/leash-BELKA/test.csv'
output_file = 'submission.csv'  # Specify the path and filename for the output file



# Read the test.csv file into a pandas DataFrame
# Wrap the loop with tqdm to display progress
for df_test in tqdm(pd.read_csv(test_file, chunksize=10_000)):
    
    # Generate ECFPs for the molecule_smiles
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)

    # One-hot encode the protein_name
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))

    # Combine ECFPs and one-hot encoded protein_name
    X_test = [ecfp + protein for ecfp, protein in zip(df_test['ecfp'].tolist(), protein_onehot.tolist())]

    # Predict the probabilities using ensemble models
    probabilities_catboost = catboost_model.predict_proba(X_test)[:, 1]
    probabilities_lgbm = lgbm_model.predict_proba(X_test)[:, 1]
    probabilities_xgb = xgb_model.predict_proba(X_test)[:, 1]

    # Average the predictions from the ensemble
    probabilities_ensemble = (probabilities_catboost + probabilities_lgbm + probabilities_xgb) / NUMBER_OF_MODELS
    
     # Append the probabilities to the list

# Create a DataFrame with 'id' and 'probability' columns
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities_ensemble})

# Save the output DataFrame to a CSV file
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))

168it [1:16:46, 27.42s/it]
